# Run benchmarking experiments
Note: The Benchmarking Framework does currently not properly support auto-regression or lagged covariates with multiple step ahead forecasts.

In [1]:
import os
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")

## Load data

In [2]:
%%capture
# create temporary data folder and download data
!rm -r tmp-data
!mkdir tmp-data
!wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/air_passengers.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/wp_log_peyton_manning.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/retail_sales.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/yosemite_temps.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/energy/ERCOT_load_2004_2021Sept.csv

In [3]:
os.listdir('tmp-data')

['air_passengers.csv']

In [4]:
air_passengers_df = pd.read_csv(os.path.join('tmp-data', 'air_passengers.csv'))
# peyton_manning_df = pd.read_csv(os.path.join('tmp-data', 'wp_log_peyton_manning.csv'))
# retail_sales_df = pd.read_csv(os.path.join('tmp-data', 'retail_sales.csv'))
# yosemite_temps_df = pd.read_csv(os.path.join('tmp-data', 'yosemite_temps.csv'))
# ercot_load_df = pd.read_csv(os.path.join('tmp-data', 'ERCOT_load_2004_2021Sept.csv'))[['ds', 'y']]

# Streamlined 

In [5]:
from neuralprophet.benchmark import Dataset, NeuralProphetModel, NeuralProphetParams, CrossValidationExperiment
from neuralprophet.benchmark import SimpleExperiment, SimpleBenchmark, CVExperiment, CVBenchmark

  

## Streamlined Simple

In [6]:
ts = Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS")
params = NeuralProphetParams(seasonality_mode="multiplicative")
metrics = ["MAE", "MSE"]
exp = SimpleExperiment(
    model_class=NeuralProphetModel,
    params=params,
    data=ts,
    metrics=metrics,
    test_percentage=25,
)

result_train, result_val = exp.fit()

TypeError: initialize() missing 1 required positional argument: 'self'

## Streamlined CV


In [ ]:
ts = Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS")
params = NeuralProphetParams(seasonality_mode="multiplicative")
metrics = ["MAE", "MSE"]
exp_cv = CrossValidationExperiment(
    model_class=NeuralProphetModel,
    params=params,
    data=ts,
    metrics=metrics,
    test_percentage=25,
    num_folds=3,
  )
result_train, result_val = exp_cv.fit()

# Streamlined Benchmark

In [ ]:
dataset_list = [
    Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS"),
    # Dataset(df = peyton_manning_df, name = "peyton_manning", freq = "D"),
    # Dataset(df = retail_sales_df, name = "retail_sales", freq = "D"),
    # Dataset(df = yosemite_temps_df, name = "yosemite_temps", freq = "5min"),
    # Dataset(df = ercot_load_df, name = "ercot_load", freq = "H"),
]

In [ ]:
params_list = [
    NeuralProphetParams(),
    NeuralProphetParams(n_changepoints= 5),
    NeuralProphetParams(seasonality_mode = "multiplicative", learning_rate = 0.1),
]


# 2 Benchmarking using Benchmark Framework

In [ ]:
from neuralprophet.benchmark import Dataset, Model, SimpleExperiment, SimpleBenchmark, CVExperiment, CVBenchmark

## 2.0 Configure Datasets and Model Parameters
First, we define the datasets that we would like to benchmerk on.

In [ ]:
datasets = [
    Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS"),
    # Dataset(df = peyton_manning_df, name = "peyton_manning", freq = "D"),
    # Dataset(df = retail_sales_df, name = "retail_sales", freq = "D"),
    # Dataset(df = yosemite_temps_df, name = "yosemite_temps", freq = "5min"),
    # Dataset(df = ercot_load_df, name = "ercot_load", freq = "H"),
]

Next, we define the models that we want to evaluate and set their hyperparameters.

In [ ]:
parameters = [
    {},
    {"n_changepoints": 5},
    {"seasonality_mode": "multiplicative", "learning_rate": 0.1},
]

## 2.2 CVBenchmark: CrossValidation Benchmarking
Setting up a crossvalidated benchmark is just as simple:

In [ ]:
experiments_cv = []
for d in datasets:
    for m in models:
        experiments_cv.append(CVExperiment(
            model=m, 
            dataset=d, 
            valid_pct=0.1, 
            n_folds=3))

In [ ]:
benchmark_cv = CVBenchmark(
    experiments = experiments_cv,
    # metrics = ['MAE', 'MSE']
)
results_summary, results_train, results_val = benchmark_cv.run()

We now also get a summary DataFrame showing the metrics' mean and standard deviation over all folds.

In [ ]:
results_summary

In [ ]:
peyton_manning = results_summary[results_summary['data'] == 'peyton_manning']
plt = peyton_manning.plot(x='params', y=['val_MAE', 'val_MAE_std'], kind='barh')

In [ ]:
air_passengers = results_summary[results_summary['data'] == 'air_passengers']
plt = air_passengers.plot(x='params', y=['val_MAE', 'val_MAE_std'], kind='barh')

The metrics for each fold are also recoreded individually:

In [ ]:
results_val

In [ ]:
# remove temporary data folder.
!rm -r tmp-data